# Single Image

In [2]:
import torch
from PIL import Image
from transformers import AutoProcessor, AutoModelForCausalLM 

device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Florence-2-large", 
    torch_dtype=torch_dtype, 
    trust_remote_code=True,
    attn_implementation="eager"
).to(device)

processor = AutoProcessor.from_pretrained("microsoft/Florence-2-large", trust_remote_code=True)

url = "../Pictures_&_Videos/Testing_Pictures/kermit.jpg" 
try:
    image = Image.open(url).convert("RGB")
except FileNotFoundError:
    raise FileNotFoundError(f"Image not found at {url}. Please check the path.")



prompt = ""
inputs = processor(text=prompt, images=image, return_tensors="pt").to(device, torch_dtype)

generated_ids = model.generate(
    input_ids=inputs["input_ids"],
    pixel_values=inputs["pixel_values"],
    max_new_tokens=4096,
    num_beams=3,
    do_sample=False,
    use_cache=False  
)

generated_text = processor.batch_decode(generated_ids, skip_special_tokens=False)[0]
parsed_answer = processor.post_process_generation(generated_text, task="<OD>", image_size=(image.width, image.height))

print(parsed_answer)

{'<OD>': {'bboxes': [[175.6160125732422, 0.7199999690055847, 797.1840209960938, 478.3199768066406]], 'labels': ['Kermit the Frog in The Muppets']}}


# Live Video Feed

In [ ]:
import torch
import cv2
from PIL import Image
from transformers import AutoProcessor, AutoModelForCausalLM


device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Florence-2-large",
    torch_dtype=torch_dtype,
    trust_remote_code=True,
    attn_implementation="eager"
).to(device)

processor = AutoProcessor.from_pretrained(
    "microsoft/Florence-2-large",
    trust_remote_code=True
)


cap = cv2.VideoCapture(0)  # 0 = default webcam

if not cap.isOpened():
    raise RuntimeError("Cannot open webcam.")

print("✅ Webcam opened. Press 'q' to quit.\n")


prompt = ""  

while True:
    ret, frame = cap.read()
    if not ret:
        print("Failed to grab frame.")
        break

    cv2.imshow("Live Camera Feed", frame)

    pil_image = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

    
    inputs = processor(
        text=prompt,
        images=pil_image,
        return_tensors="pt"
    ).to(device, torch_dtype)

    
    generated_ids = model.generate(
        input_ids=inputs["input_ids"],
        pixel_values=inputs["pixel_values"],
        max_new_tokens=512,
        num_beams=3,
        do_sample=False,
        use_cache=False
    )

    
    generated_text = processor.batch_decode(
        generated_ids, 
        skip_special_tokens=False
    )[0]

    parsed_answer = processor.post_process_generation(
        generated_text,
        task="<OD>",   
        image_size=(pil_image.width, pil_image.height)
    )

    print("\n--- NEW FRAME ---")
    print(parsed_answer['<OD>']['labels'])

    # Quit with q
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


d:\Anaconda\envs\HCI_LAB\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
`torch_dtype` is deprecated! Use `dtype` instead!


✅ Webcam opened. Press 'q' to quit.


--- NEW FRAME ---
['man with beard and mustache in dark room with light shining on him', 'woman wearing headscarf in dimly lit room']

--- NEW FRAME ---
['woman in military uniform with headscarf and scarf']

--- NEW FRAME ---
['woman in headscarf and military uniform in room with air conditioner', 'man']

--- NEW FRAME ---
['woman in military uniform with headscarf and scarf']

--- NEW FRAME ---
['air conditioner']

--- NEW FRAME ---
['air conditioner']

--- NEW FRAME ---
['woman in military uniform smiling in front of air conditioning unit', 'air conditioner']

--- NEW FRAME ---
['woman in military uniform with headscarf smiling at camera']

--- NEW FRAME ---
['man in military uniform with headscarf smiling at camera']

--- NEW FRAME ---
['man in military uniform with white cap and gray shirt']

--- NEW FRAME ---
['person']

--- NEW FRAME ---
['man with beard and mustache smiling in front of window with curtains']

--- NEW FRAME ---
['man with be